In [1]:
from spm import * 

import numpy as np
import os.path as op
import os 
import scipy.io as sio

```
% Directory containing the attention data
%--------------------------------------------------------------------------
data_path = fileparts(mfilename('fullpath'));
if isempty(data_path), data_path = pwd; end
fprintf('%-40s:', 'Downloading Attention dataset...');
urlwrite('http://www.fil.ion.ucl.ac.uk/spm/download/data/attention/attention.zip','attention.zip');
unzip(fullfile(data_path,'attention.zip'));
data_path = fullfile(data_path,'attention');
fprintf(' %30s\n', '...done');
```

In [2]:
data_path = op.join('data', 'attention')
zip_path  = op.join('data', 'attention.zip')
if not op.isdir(data_path):
    if not op.isfile(zip_path):
        import wget
        wget.download('https://www.fil.ion.ucl.ac.uk/spm/download/data/attention/attention.zip', 'data')

    import shutil
    shutil.unpack_archive(zip_path, 'data', 'zip')

```
% Initialise SPM
%--------------------------------------------------------------------------
spm('Defaults','fMRI');
spm_jobman('initcfg');
```

In [3]:
spm('Defaults','fMRI')
spm_jobman('initcfg', nargout=0)

Initializing Matlab Runtime...


```
% SPECIFICATION DCMs "attentional modulation of backward/forward connection"
%--------------------------------------------------------------------------
% To specify a DCM, you might want to create a template one using the GUI
% then use spm_dcm_U.m and spm_dcm_voi.m to insert new inputs and new
% regions. The following code creates a DCM file from scratch, which
% involves some technical subtleties and a deeper knowledge of the DCM
% structure.

load(fullfile(data_path,'GLM','SPM.mat'));
```

In [4]:
SPM = Runtime.call('load', op.join(data_path, 'GLM', 'SPM.mat'))['SPM']

In [5]:
DCM = Struct() 

xY1 = Runtime.call('load', op.join(data_path,'GLM','VOI_V1_1.mat'))['xY'];
xY2 = Runtime.call('load', op.join(data_path,'GLM','VOI_V5_1.mat'))['xY'];
xY3 = Runtime.call('load', op.join(data_path,'GLM','VOI_SPC_1.mat'))['xY'];

DCM.xY = StructArray(xY1, xY2, xY3)

In [6]:
DCM.n = 3;     
DCM.v = xY1['u'].shape[0]; 

```
% Time series
%--------------------------------------------------------------------------
DCM.Y.dt  = SPM.xY.RT;
DCM.Y.X0  = DCM.xY(1).X0;
for i = 1:DCM.n
    DCM.Y.y(:,i)  = DCM.xY(i).u;
    DCM.Y.name{i} = DCM.xY(i).name;
end

DCM.Y.Q    = spm_Ce(ones(1,DCM.n)*DCM.v);
```

In [7]:
DCM.Y = Struct()
DCM.Y.dt  = SPM.xY.RT;
DCM.Y.X0  = xY1.X0;

DCM.Y.y = np.concatenate([xY.u for xY in (xY1, xY2, xY3)], axis=1)
DCM.Y.name = [xY.name for xY in  (xY1, xY2, xY3)]

DCM.Y.Q    = spm_Ce(np.ones((1,DCM.n))*DCM.v);

```
% Experimental inputs
%--------------------------------------------------------------------------
DCM.U.dt   =  SPM.Sess.U(1).dt;
DCM.U.name = [SPM.Sess.U.name];
DCM.U.u    = [SPM.Sess.U(1).u(33:end,1) ...
              SPM.Sess.U(2).u(33:end,1) ...
              SPM.Sess.U(3).u(33:end,1)];
```

In [8]:
DCM.U = Struct()
DCM.U.dt   =  SPM.Sess.U[0].dt;
DCM.U.name = [u.name for u in SPM.Sess.U];
DCM.U.u    = np.concatenate([
        u.u[32:] for u in SPM.Sess.U
    ], axis=1);

```
% DCM parameters and options
%--------------------------------------------------------------------------
DCM.delays = repmat(SPM.xY.RT/2,DCM.n,1);
DCM.TE     = 0.04;

DCM.options.nonlinear  = 0;
DCM.options.two_state  = 0;
DCM.options.stochastic = 0;
DCM.options.nograph    = 1;
```

In [9]:
DCM.delays = np.repeat([[SPM['xY']['RT']/2]], DCM.n, 1);
DCM.TE     = 0.04;

DCM.options = Struct()
DCM.options.nonlinear  = 0;
DCM.options.two_state  = 0;
DCM.options.stochastic = 0;
DCM.options.nograph    = 1;

```
% Connectivity matrices for model with backward modulation
%--------------------------------------------------------------------------
DCM.a = [1 1 0; 1 1 1; 0 1 1];
DCM.b = zeros(3,3,3);  DCM.b(2,1,2) = 1;  DCM.b(2,3,3) = 1;
DCM.c = [1 0 0; 0 0 0; 0 0 0];
DCM.d = zeros(3,3,0);

save(fullfile(data_path,'GLM','DCM_mod_bwd.mat'),'DCM');
```

In [10]:
DCM.a = np.array([[1, 1, 0], [1, 1, 1], [0, 1, 1]])
DCM.b = np.zeros((3,3,3));  
DCM.b[1,0,1] = 1;  
DCM.b[1,2,2] = 1;
DCM.c = np.array([[1, 0, 0], [0, 0, 0], [0, 0, 0]])
DCM.d = np.zeros((3,3,0));

DCMbwd = spm_dcm_estimate(DCM)

```
% Connectivity matrices for model with forward modulation
%--------------------------------------------------------------------------
DCM.b = zeros(3,3,3);  DCM.b(2,1,2) = 1;  DCM.b(2,1,3) = 1;

save(fullfile(data_path,'GLM','DCM_mod_fwd.mat'),'DCM');
```

In [11]:
DCM.b = np.zeros((3,3,3));  
DCM.b[1,0,1] = 1;  
DCM.b[1,0,2] = 1;

DCMfwd = spm_dcm_estimate(DCM)

```
% Bayesian Model Comparison
% --------------------------------------------------------------------------
DCM_bwd = load('DCM_mod_bwd.mat','F');
DCM_fwd = load('DCM_mod_fwd.mat','F');
fprintf('Model evidence: %f (bwd) vs %f (fwd)\n',DCMbwd.F,DCMfwd.F);
```

In [12]:
print(f"Model evidence: {DCMbwd['F']} (bwd) vs {DCMfwd['F']} (fwd)")

Model evidence: -3441.403050497822 (bwd) vs -3422.1295455880836 (fwd)
